In [5]:
import pandas as pd
df_data = pd.read_excel("Full_Data_Result_tekil.xlsx")
df_data

,Tarih,customId,id,startTimestamp,tournament_name,tournament_category_name,season_name,season_year,tournament_uniqueTournament_id,season_id,...,homeTeam_id,awayTeam_name,awayTeam_nameCode,awayTeam_id,homeScore_normaltime,homeScore_period1,homeScore_period2,awayScore_normaltime,awayScore_period1,awayScore_period2
0,2024-01-01,UzpsNlTd,11408889,2023-12-31 14:00:00,Liga Portugal 2,Portugal,Liga Portugal 2 23/24,23/24,239,52770.0,...,38744,AVS - Futebol SAD,AVS,483088,0.0,0.0,0.0,2.0,2.0,0.0
1,2024-01-01,jzwsuuic,11909722,2023-12-31 14:30:00,Pro League,Iran,Pro League 23/24,23/24,915,52957.0,...,270969,Malavan Bandar Anzali FC,MBA,53709,1.0,1.0,0.0,1.0,1.0,0.0
2,2024-01-01,yQTspYrc,11909725,2023-12-31 14:30:00,Pro League,Iran,Pro League 23/24,23/24,915,52957.0,...,294915,Esteghlal Khuzestan,ETK,109573,2.0,0.0,2.0,0.0,0.0,0.0
3,2024-01-01,jYbsfat,11362403,2023-12-31 15:30:00,Cymru Premier,Wales,Cymru Premier 23/24,23/24,254,52310.0,...,4909,Colwyn Bay,COL,45005,2.0,0.0,2.0,1.0,0.0,1.0
4,2024-01-01,eddsETQb,11830313,2023-12-31 16:30:00,Premiership,South Africa,Premiership 23/24,23/24,358,53301.0,...,7654,Cape Town City FC,CTC,229679,0.0,0.0,0.0,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46990,2024-11-28,aeusUvjc,13119462,2024-11-28 03:30:00,"Division Profesional, Clausura",Bolivia,División Profesional 2024,2024,16736,58156.0,...,273544,Nacional Potosí,CNP,47700,2.0,1.0,1.0,0.0,0.0,0.0
46991,2024-11-28,GNsON,13158192,2024-11-28 04:00:00,"Liga MX, Apertura Playoffs",Mexico,"Liga MX, Apertura 2024",2024,11621,61419.0,...,1939,CD Toluca,TOL,1931,2.0,0.0,2.0,0.0,0.0,0.0
46992,2024-11-28,fxcstxc,13149323,2024-11-28 04:30:00,"Primera A, Clausura, Championship, Group B",Colombia,"Primera A, Finalización 2024",2024,11536,63819.0,...,6118,Junior Barranquilla,JUN,6105,2.0,2.0,0.0,2.0,1.0,1.0
46993,2024-11-28,FedslEj,13169958,2024-11-28 05:30:00,"Liga Guate Banrural, Apertura Playoffs",Guatemala,Liga Nacional. Apertura 2024,2024,11619,61855.0,...,23961,Deportivo Malacateco,MAL,7730,2.0,0.0,2.0,0.0,0.0,0.0


In [9]:
import pandas as pd


def FTResult(df_data):

    # 1. Ev ve deplasman takımları için attığı ve yediği golleri hesapla
    df_data['home_goals_first_half'] = df_data['homeScore_period1']
    df_data['home_goals_second_half'] = df_data['homeScore_period2']
    df_data['away_goals_first_half'] = df_data['awayScore_period1']
    df_data['away_goals_second_half'] = df_data['awayScore_period2']
    
    df_data['home_total_goals'] = df_data['home_goals_first_half'] + df_data['home_goals_second_half']
    df_data['away_total_goals'] = df_data['away_goals_first_half'] + df_data['away_goals_second_half']
    
    df_data['home_goals_conceded'] = df_data['away_total_goals']
    df_data['away_goals_conceded'] = df_data['home_total_goals']
    
    # 2. Maç sonucu için durum belirle (ev sahibi için)
    df_data['home_result'] = df_data['home_total_goals'] - df_data['away_total_goals']
    df_data['away_result'] = -df_data['home_result']
    
    # 3. Her takımın ev ve deplasman performansını hesapla
    # Ev sahibi için
    home_performance = df_data.groupby(['season_id', 'homeTeam_id']).agg(
        matches_played=('homeTeam_id', 'count'),
        wins=('home_result', lambda x: (x > 0).sum()),
        draws=('home_result', lambda x: (x == 0).sum()),
        losses=('home_result', lambda x: (x < 0).sum()),
        goals_scored=('home_total_goals', 'sum'),
        goals_conceded=('home_goals_conceded', 'sum')
    ).reset_index()

    home_performance.rename(columns={'homeTeam_id': 'team_id'}, inplace=True)
    home_performance['location'] = 'home'
    
    # Deplasman için
    away_performance = df_data.groupby(['season_id', 'awayTeam_id']).agg(
        matches_played=('awayTeam_id', 'count'),
        wins=('away_result', lambda x: (x > 0).sum()),
        draws=('away_result', lambda x: (x == 0).sum()),
        losses=('away_result', lambda x: (x < 0).sum()),
        goals_scored=('away_total_goals', 'sum'),
        goals_conceded=('away_goals_conceded', 'sum')
    ).reset_index()
    
    away_performance.rename(columns={'awayTeam_id': 'team_id'}, inplace=True)
    away_performance['location'] = 'away'
    
    # 4. Ev ve deplasman performanslarını birleştir
    team_performance = pd.concat([home_performance, away_performance], ignore_index=True)
    return team_performance



In [3]:
team_performance.to_excel("Team_Performance.xlsx")

In [10]:
import pandas as pd


def HTResult():

    # 1. İlk yarıda attığı ve yediği golleri hesapla
    df_data['home_goals_first_half'] = df_data['homeScore_period1']
    df_data['away_goals_first_half'] = df_data['awayScore_period1']
    
    df_data['home_goals_conceded_first_half'] = df_data['away_goals_first_half']
    df_data['away_goals_conceded_first_half'] = df_data['home_goals_first_half']
    
    # 2. İlk yarı sonucu için durum belirle (ev sahibi için)
    df_data['home_result_first_half'] = df_data['home_goals_first_half'] - df_data['away_goals_first_half']
    df_data['away_result_first_half'] = -df_data['home_result_first_half']
    
    # 3. Ev ve deplasman performansını hesapla (sadece ilk yarı için)
    # Ev sahibi için
    home_performance = df_data.groupby(['season_id', 'homeTeam_id']).agg(
        matches_played=('homeTeam_id', 'count'),
        wins=('home_result_first_half', lambda x: (x > 0).sum()),
        draws=('home_result_first_half', lambda x: (x == 0).sum()),
        losses=('home_result_first_half', lambda x: (x < 0).sum()),
        goals_scored=('home_goals_first_half', 'sum'),
        goals_conceded=('home_goals_conceded_first_half', 'sum')
    ).reset_index()
    
    home_performance.rename(columns={'homeTeam_id': 'team_id'}, inplace=True)
    home_performance['location'] = 'home'

    # Deplasman için
    away_performance = df_data.groupby(['season_id', 'awayTeam_id']).agg(
        matches_played=('awayTeam_id', 'count'),
        wins=('away_result_first_half', lambda x: (x > 0).sum()),
        draws=('away_result_first_half', lambda x: (x == 0).sum()),
        losses=('away_result_first_half', lambda x: (x < 0).sum()),
        goals_scored=('away_goals_first_half', 'sum'),
        goals_conceded=('away_goals_conceded_first_half', 'sum')
    ).reset_index()
    
    away_performance.rename(columns={'awayTeam_id': 'team_id'}, inplace=True)
    away_performance['location'] = 'away'
    
    # 4. Ev ve deplasman performanslarını birleştir
    team_performance2 = pd.concat([home_performance, away_performance], ignore_index=True)
    
    return team_performance2


In [11]:
import pandas as pd

# Fonksiyon: Genel ve ilk yarı performanslarını hesapla
def calculate_team_performance(df):
    # 1. Gerekli verileri hesaplama
    # Genel sonuçlar
    df['home_goals_first_half'] = df['homeScore_period1']
    df['home_goals_second_half'] = df['homeScore_period2']
    df['away_goals_first_half'] = df['awayScore_period1']
    df['away_goals_second_half'] = df['awayScore_period2']

    df['home_total_goals'] = df['home_goals_first_half'] + df['home_goals_second_half']
    df['away_total_goals'] = df['away_goals_first_half'] + df['away_goals_second_half']

    df['home_goals_conceded'] = df['away_total_goals']
    df['away_goals_conceded'] = df['home_total_goals']

    df['home_result'] = df['home_total_goals'] - df['away_total_goals']
    df['away_result'] = -df['home_result']

    # İlk yarı sonuçları
    df['home_result_first_half'] = df['home_goals_first_half'] - df['away_goals_first_half']
    df['away_result_first_half'] = -df['home_result_first_half']
    df['home_goals_conceded_first_half'] = df['away_goals_first_half']
    df['away_goals_conceded_first_half'] = df['home_goals_first_half']

    # 2. Performans hesaplama
    # Ev performansı
    home_performance = df.groupby(['season_id', 'homeTeam_id']).agg(
        matches_played=('homeTeam_id', 'count'),
        wins=('home_result', lambda x: (x > 0).sum()),
        draws=('home_result', lambda x: (x == 0).sum()),
        losses=('home_result', lambda x: (x < 0).sum()),
        goals_scored=('home_total_goals', 'sum'),
        goals_conceded=('home_goals_conceded', 'sum'),
        wins_first_half=('home_result_first_half', lambda x: (x > 0).sum()),
        draws_first_half=('home_result_first_half', lambda x: (x == 0).sum()),
        losses_first_half=('home_result_first_half', lambda x: (x < 0).sum()),
        goals_scored_first_half=('home_goals_first_half', 'sum'),
        goals_conceded_first_half=('home_goals_conceded_first_half', 'sum')
    ).reset_index()

    home_performance.rename(columns={'homeTeam_id': 'team_id'}, inplace=True)
    home_performance['location'] = 'home'

    # Deplasman performansı
    away_performance = df.groupby(['season_id', 'awayTeam_id']).agg(
        matches_played=('awayTeam_id', 'count'),
        wins=('away_result', lambda x: (x > 0).sum()),
        draws=('away_result', lambda x: (x == 0).sum()),
        losses=('away_result', lambda x: (x < 0).sum()),
        goals_scored=('away_total_goals', 'sum'),
        goals_conceded=('away_goals_conceded', 'sum'),
        wins_first_half=('away_result_first_half', lambda x: (x > 0).sum()),
        draws_first_half=('away_result_first_half', lambda x: (x == 0).sum()),
        losses_first_half=('away_result_first_half', lambda x: (x < 0).sum()),
        goals_scored_first_half=('away_goals_first_half', 'sum'),
        goals_conceded_first_half=('away_goals_conceded_first_half', 'sum')
    ).reset_index()

    away_performance.rename(columns={'awayTeam_id': 'team_id'}, inplace=True)
    away_performance['location'] = 'away'

    # 3. Performansları birleştirme
    team_performance = pd.concat([home_performance, away_performance], ignore_index=True)

    return team_performance


# Performansı hesapla
team_performance = calculate_team_performance(df_data)

team_performance


,season_id,team_id,matches_played,wins,draws,losses,goals_scored,goals_conceded,wins_first_half,draws_first_half,losses_first_half,goals_scored_first_half,goals_conceded_first_half,location
0,40128.0,4776,1,1,0,0,1.0,0.0,0,1,0,0.0,0.0,home
1,40128.0,129264,1,0,0,1,0.0,1.0,0,1,0,0.0,0.0,home
2,46599.0,4479,1,0,0,1,1.0,2.0,0,1,0,0.0,0.0,home
3,46599.0,4480,1,0,0,1,1.0,4.0,0,0,1,1.0,2.0,home
4,46599.0,4701,1,1,0,0,2.0,1.0,0,0,1,0.0,1.0,home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18567,68433.0,387698,1,0,0,1,0.0,2.0,0,0,1,0.0,2.0,away
18568,68433.0,451441,1,0,0,1,0.0,1.0,0,1,0,0.0,0.0,away
18569,68841.0,1780,1,0,0,1,2.0,4.0,0,1,0,2.0,2.0,away
18570,68841.0,199312,1,0,0,1,0.0,3.0,0,0,1,0.0,1.0,away


In [19]:
import pandas as pd

def calculate_team_performance_with_ratios(df):
    # 1. Gerekli verileri hesaplama
    # Genel sonuçlar

    
    df['home_goals_first_half'] = df['homeScore_period1']
    df['home_goals_second_half'] = df['homeScore_period2']
    df['away_goals_first_half'] = df['awayScore_period1']
    df['away_goals_second_half'] = df['awayScore_period2']

    df['home_total_goals'] = df['home_goals_first_half'] + df['home_goals_second_half']
    df['away_total_goals'] = df['away_goals_first_half'] + df['away_goals_second_half']

    df['home_goals_conceded'] = df['away_total_goals']
    df['away_goals_conceded'] = df['home_total_goals']

    df['home_result'] = df['home_total_goals'] - df['away_total_goals']
    df['away_result'] = -df['home_result']

    # İlk yarı sonuçları
    df['home_result_first_half'] = df['home_goals_first_half'] - df['away_goals_first_half']
    df['away_result_first_half'] = -df['home_result_first_half']
    df['home_goals_conceded_first_half'] = df['away_goals_first_half']
    df['away_goals_conceded_first_half'] = df['home_goals_first_half']

    # 2. Performans hesaplama
    # Ev performansı
    home_performance = df.groupby(['season_id', 'homeTeam_id','homeTeam_name',"season_name","tournament_name","tournament_category_name",
                                  "season_year","tournament_uniqueTournament_id"]).agg(
        matches_played=('homeTeam_id', 'count'),
        wins=('home_result', lambda x: (x > 0).sum()),
        draws=('home_result', lambda x: (x == 0).sum()),
        losses=('home_result', lambda x: (x < 0).sum()),
        goals_scored=('home_total_goals', 'sum'),
        goals_conceded=('home_goals_conceded', 'sum'),
        wins_first_half=('home_result_first_half', lambda x: (x > 0).sum()),
        draws_first_half=('home_result_first_half', lambda x: (x == 0).sum()),
        losses_first_half=('home_result_first_half', lambda x: (x < 0).sum()),
        goals_scored_first_half=('home_goals_first_half', 'sum'),
        goals_conceded_first_half=('home_goals_conceded_first_half', 'sum')
    ).reset_index()

    home_performance.rename(columns={'homeTeam_id': 'team_id'}, inplace=True)
    home_performance.rename(columns={'homeTeam_name': 'team_name'}, inplace=True)
    home_performance['location'] = 'home'

    # Deplasman performansı
    away_performance = df.groupby(['season_id', 'awayTeam_id','awayTeam_name',"season_name","tournament_name","tournament_category_name",
                                  "season_year","tournament_uniqueTournament_id"]).agg(
        matches_played=('awayTeam_id', 'count'),
        wins=('away_result', lambda x: (x > 0).sum()),
        draws=('away_result', lambda x: (x == 0).sum()),
        losses=('away_result', lambda x: (x < 0).sum()),
        goals_scored=('away_total_goals', 'sum'),
        goals_conceded=('away_goals_conceded', 'sum'),
        wins_first_half=('away_result_first_half', lambda x: (x > 0).sum()),
        draws_first_half=('away_result_first_half', lambda x: (x == 0).sum()),
        losses_first_half=('away_result_first_half', lambda x: (x < 0).sum()),
        goals_scored_first_half=('away_goals_first_half', 'sum'),
        goals_conceded_first_half=('away_goals_conceded_first_half', 'sum')
    ).reset_index()

    away_performance.rename(columns={'awayTeam_id': 'team_id'}, inplace=True)
    away_performance.rename(columns={'awayTeam_name': 'team_name'}, inplace=True)
    away_performance['location'] = 'away'

    # 3. Performansları birleştirme
    team_performance = pd.concat([home_performance, away_performance], ignore_index=True)

    # 4. Oranları ve ortalamaları hesaplama
    team_performance['win_rate'] = team_performance['wins'] / team_performance['matches_played']
    team_performance['draw_rate'] = team_performance['draws'] / team_performance['matches_played']
    team_performance['loss_rate'] = team_performance['losses'] / team_performance['matches_played']
    team_performance['goals_scored_per_match'] = team_performance['goals_scored'] / team_performance['matches_played']
    team_performance['goals_conceded_per_match'] = team_performance['goals_conceded'] / team_performance['matches_played']

    team_performance['win_rate_first_half'] = team_performance['wins_first_half'] / team_performance['matches_played']
    team_performance['draw_rate_first_half'] = team_performance['draws_first_half'] / team_performance['matches_played']
    team_performance['loss_rate_first_half'] = team_performance['losses_first_half'] / team_performance['matches_played']
    team_performance['goals_scored_first_half_per_match'] = team_performance['goals_scored_first_half'] / team_performance['matches_played']
    team_performance['goals_conceded_first_half_per_match'] = team_performance['goals_conceded_first_half'] / team_performance['matches_played']

    return team_performance

# Performansı hesapla
team_performance = calculate_team_performance_with_ratios(df_data)

team_performance

,season_id,team_id,team_name,season_name,tournament_name,tournament_category_name,season_year,tournament_uniqueTournament_id,matches_played,wins,...,win_rate,draw_rate,loss_rate,goals_scored_per_match,goals_conceded_per_match,win_rate_first_half,draw_rate_first_half,loss_rate_first_half,goals_scored_first_half_per_match,goals_conceded_first_half_per_match
0,40128.0,4776,Lithuania,UEFA Nations League 22/23,"UEFA Nations League, Promotion/Relegation Play...",Europe,2022/2023,10783,1,1,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,40128.0,129264,Gibraltar,UEFA Nations League 22/23,"UEFA Nations League, Promotion/Relegation Play...",Europe,2022/2023,10783,1,0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,46599.0,4479,Bosnia & Herzegovina,"European Championship 2024, Qualification","Euro, Qualification Playoff",Europe,2024,27,1,0,...,0.0,0.0,1.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0
3,46599.0,4480,Israel,"European Championship 2024, Qualification","Euro, Qualification Playoff",Europe,2024,27,1,0,...,0.0,0.0,1.0,1.0,4.0,0.0,0.0,1.0,1.0,2.0
4,46599.0,4701,Ukraine,"European Championship 2024, Qualification","Euro, Qualification Playoff",Europe,2024,27,1,1,...,1.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20942,68433.0,387698,Rajasthan United FC,I-League 24/25,I-League,India,24/25,848,1,0,...,0.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0
20943,68433.0,451441,SC Bengaluru,I-League 24/25,I-League,India,24/25,848,1,0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
20944,68841.0,1780,Enköpings SK,"Ettan, Relegation/Promotion 2024","Ettan, Relegation/Promotion",Sweden,2024,11617,1,0,...,0.0,0.0,1.0,2.0,4.0,0.0,1.0,0.0,2.0,2.0
20945,68841.0,199312,BK Astrio,"Ettan, Relegation/Promotion 2024","Ettan, Relegation/Promotion",Sweden,2024,11617,1,0,...,0.0,0.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0


In [20]:
team_performance.to_excel("TeamPerformace_with_ratio.xlsx")

In [14]:
df_data.columns

Index(['Tarih', 'customId', 'id', 'startTimestamp', 'tournament_name',
       'tournament_category_name', 'season_name', 'season_year',
       'tournament_uniqueTournament_id', 'season_id', 'roundInfo_round',
       'status_type', 'homeTeam_name', 'homeTeam_nameCode', 'homeTeam_id',
       'awayTeam_name', 'awayTeam_nameCode', 'awayTeam_id',
       'homeScore_normaltime', 'homeScore_period1', 'homeScore_period2',
       'awayScore_normaltime', 'awayScore_period1', 'awayScore_period2',
       'home_goals_first_half', 'away_goals_first_half',
       'home_goals_conceded_first_half', 'away_goals_conceded_first_half',
       'home_result_first_half', 'away_result_first_half',
       'home_goals_second_half', 'away_goals_second_half', 'home_total_goals',
       'away_total_goals', 'home_goals_conceded', 'away_goals_conceded',
       'home_result', 'away_result'],
      dtype='object')

In [21]:
def calculate_2_to_1_probability(df, home_id, away_id):
    # 1. İlgili takımların verilerini seç
    home_team = df[df['team_id'] == home_id].iloc[0]
    away_team = df[df['team_id'] == away_id].iloc[0]
    
    # 2. İlk yarı galibiyet, beraberlik ve mağlubiyet oranlarını hesapla
    away_first_half_win_rate = away_team['win_rate_first_half']  # Deplasmanın ilk yarıda kazanma oranı
    home_second_half_win_rate = home_team['win_rate']           # Ev sahibinin maç kazanma oranı (genel)

    # 3. "2'den 1'e" olasılık hesaplama
    probability_2_to_1 = away_first_half_win_rate * home_second_half_win_rate

    # 4. Sonuçları döndür
    return {
        "home_team": home_team['team_name'],
        "away_team": away_team['team_name'],
        "away_first_half_win_rate": away_first_half_win_rate,
        "home_second_half_win_rate": home_second_half_win_rate,
        "probability_2_to_1": probability_2_to_1
    }

# Örnek: Lithuania (home_id: 4776) ve Gibraltar (away_id: 129264)
result = calculate_2_to_1_probability(team_performance, home_id=3052, away_id=3061)

# Sonuçları yazdır
result


{'home_team': 'Fenerbahçe',
 'away_team': 'Galatasaray',
 'away_first_half_win_rate': 0.6363636363636364,
 'home_second_half_win_rate': 0.5,
 'probability_2_to_1': 0.3181818181818182}

In [22]:
fiksture_df = pd.read_excel("29KasımFiksture.xlsx")
fiksture_df

,Unnamed: 0,Tarih,customId,id,startTimestamp,tournament_name,tournament_category_name,season_name,season_year,tournament_uniqueTournament_id,...,homeTeam_id,awayTeam_name,awayTeam_nameCode,awayTeam_id,homeScore_normaltime,homeScore_period1,homeScore_period2,awayScore_normaltime,awayScore_period1,awayScore_period2
0,0,2024-11-29,FV,12436501,2024-11-29 23:00:00,Premier League,England,Premier League 24/25,24/25,17,...,30,Southampton,SOU,45,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2024-11-29,BgbsDgb,12437710,2024-11-29 23:00:00,LaLiga,Spain,LaLiga 24/25,24/25,8,...,2826,Valencia,VCF,2828,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2024-11-29,rgbsCGc,12437708,2024-11-30 16:00:00,LaLiga,Spain,LaLiga 24/25,24/25,8,...,2817,Las Palmas,LPA,6577,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2024-11-29,bebsueb,12504678,2024-11-29 22:45:00,Serie A,Italy,Serie A 24/25,24/25,23,...,2719,Hellas Verona,VER,2701,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2024-11-29,Babsybb,12499468,2024-11-29 22:30:00,Bundesliga,Germany,Bundesliga 24/25,24/25,35,...,2526,Holstein Kiel,HOK,2573,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,279,2024-11-29,mYcszPi,13171706,2024-11-29 21:00:00,"International Friendly Games, Women",World,Int. Friendly Games W 2024,2024,852,...,7412,Iceland,ISL,22024,NaN,NaN,NaN,NaN,NaN,NaN
280,280,2024-11-29,xNiswXo,13171705,2024-11-29 21:00:00,"International Friendly Games, Women",World,Int. Friendly Games W 2024,2024,852,...,37371,South Korea,KOR,21922,NaN,NaN,NaN,NaN,NaN,NaN
281,281,2024-11-29,REjsdmp,13168654,2024-11-29 22:00:00,"International Friendly Games, Women",World,Int. Friendly Games W 2024,2024,852,...,38103,Germany,GER,23992,NaN,NaN,NaN,NaN,NaN,NaN
282,282,2024-11-29,nYcsJZm,13156380,2024-11-29 22:45:00,"International Friendly Games, Women",World,Int. Friendly Games W 2024,2024,852,...,32484,China,CHN,7413,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:

def calculate_fixtures_probability(fixture_df, performance_df):
    probabilities = []
    home_team_details = []
    away_team_details = []

    for _, row in fixture_df.iterrows():
        home_id = row['homeTeam_id']
        away_id = row['awayTeam_id']

        # İlgili takımları performans verisinden seç
        home_team = performance_df[performance_df['team_id'] == home_id]
        away_team = performance_df[performance_df['team_id'] == away_id]

        # Eğer takımlardan biri performans datasında yoksa oran 0 olarak eklenir
        if home_team.empty or away_team.empty:
            probabilities.append(0.0)
            home_team_details.append(None)
            away_team_details.append(None)
            continue

        home_team = home_team.iloc[0]
        away_team = away_team.iloc[0]

        # İlk yarı ve ikinci yarı oranlarını al
        away_first_half_win_rate = away_team['win_rate_first_half']
        home_second_half_win_rate = home_team['win_rate']

        # 2'den 1'e bitme olasılığını hesapla
        probability_2_to_1 = away_first_half_win_rate * home_second_half_win_rate
        probabilities.append(probability_2_to_1)

        # Tüm sayısal bilgileri ekle
        home_team_details.append(home_team.to_dict())
        away_team_details.append(away_team.to_dict())

    # Oranları fikstür DataFrame'ine ekle
    fixture_df['probability_2_to_1'] = probabilities
    fixture_df['home_team_details'] = home_team_details
    fixture_df['away_team_details'] = away_team_details

    # Home ve Away detaylarını DataFrame'e genişletme
    home_details_df = pd.json_normalize(fixture_df['home_team_details'])
    away_details_df = pd.json_normalize(fixture_df['away_team_details'])

    # Yeni detayları ana fikstür DataFrame'e ekle
    fixture_df = pd.concat([fixture_df, home_details_df.add_prefix('home_'), away_details_df.add_prefix('away_')], axis=1)

    # Detay sütunları temizlenebilir
    fixture_df.drop(columns=['home_team_details', 'away_team_details'], inplace=True)

    return fixture_df



In [27]:
data_result = calculate_fixtures_probability(fiksture_df,team_performance)
data_result

,Unnamed: 0,Tarih,customId,id,startTimestamp,tournament_name,tournament_category_name,season_name,season_year,tournament_uniqueTournament_id,...,away_win_rate,away_draw_rate,away_loss_rate,away_goals_scored_per_match,away_goals_conceded_per_match,away_win_rate_first_half,away_draw_rate_first_half,away_loss_rate_first_half,away_goals_scored_first_half_per_match,away_goals_conceded_first_half_per_match
0,0,2024-11-29,FV,12436501,2024-11-29 23:00:00,Premier League,England,Premier League 24/25,24/25,17,...,0.600000,0.100000,0.300000,2.40,1.400000,0.600000,0.000000,0.400000,1.200000,0.800000
1,1,2024-11-29,BgbsDgb,12437710,2024-11-29 23:00:00,LaLiga,Spain,LaLiga 24/25,24/25,8,...,0.363636,0.363636,0.272727,1.00,0.909091,0.363636,0.454545,0.181818,0.636364,0.272727
2,2,2024-11-29,rgbsCGc,12437708,2024-11-30 16:00:00,LaLiga,Spain,LaLiga 24/25,24/25,8,...,0.181818,0.272727,0.545455,1.00,1.363636,0.181818,0.454545,0.363636,0.272727,0.454545
3,3,2024-11-29,bebsueb,12504678,2024-11-29 22:45:00,Serie A,Italy,Serie A 24/25,24/25,23,...,0.400000,0.300000,0.300000,1.40,1.400000,0.200000,0.700000,0.100000,0.700000,0.600000
4,4,2024-11-29,Babsybb,12499468,2024-11-29 22:30:00,Bundesliga,Germany,Bundesliga 24/25,24/25,35,...,0.500000,0.125000,0.375000,1.75,1.250000,0.375000,0.375000,0.250000,0.750000,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,279,2024-11-29,mYcszPi,13171706,2024-11-29 21:00:00,"International Friendly Games, Women",World,Int. Friendly Games W 2024,2024,852,...,1.000000,0.000000,0.000000,2.00,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
280,280,2024-11-29,xNiswXo,13171705,2024-11-29 21:00:00,"International Friendly Games, Women",World,Int. Friendly Games W 2024,2024,852,...,1.000000,0.000000,0.000000,2.50,0.500000,0.500000,0.500000,0.000000,1.000000,0.000000
281,281,2024-11-29,REjsdmp,13168654,2024-11-29 22:00:00,"International Friendly Games, Women",World,Int. Friendly Games W 2024,2024,852,...,0.000000,0.000000,1.000000,1.00,2.000000,0.000000,1.000000,0.000000,1.000000,1.000000
282,282,2024-11-29,nYcsJZm,13156380,2024-11-29 22:45:00,"International Friendly Games, Women",World,Int. Friendly Games W 2024,2024,852,...,1.000000,0.000000,0.000000,2.50,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000


In [28]:
data_result.to_excel("Prob_2_1.xlsx")